# Generate list of Extension in CSV

In [1]:
from modules.utils import get_installed_extensions, load_environment_variables, generate_extension_data
from modules.save_as import save_to_csv, save_to_markdown
dotenv_path = '.env'

In [2]:
code_cmd_path = load_environment_variables(dotenv_path)
extensions = get_installed_extensions(code_cmd_path)
print(extensions)

['be5invis.vscode-custom-css', 'bierner.github-markdown-preview', 'bierner.markdown-checkbox', 'bierner.markdown-emoji', 'bierner.markdown-footnotes', 'bierner.markdown-mermaid', 'bierner.markdown-preview-github-styles', 'bierner.markdown-yaml-preamble', 'chadalen.vscode-jetbrains-icon-theme', 'davidanson.vscode-markdownlint', 'eamodio.gitlens', 'eliostruyf.vscode-front-matter', 'esbenp.prettier-vscode', 'github.copilot', 'github.copilot-chat', 'github.github-vscode-theme', 'golang.go', 'jock.svg', 'khaeransori.json2csv', 'meezilla.json', 'miguelsolorio.fluent-icons', 'ms-python.debugpy', 'ms-python.python', 'ms-python.vscode-pylance', 'ms-toolsai.datawrangler', 'ms-toolsai.jupyter', 'ms-toolsai.jupyter-keymap', 'ms-toolsai.jupyter-renderers', 'ms-toolsai.python-ds-extension-pack', 'ms-toolsai.vscode-jupyter-cell-tags', 'ms-toolsai.vscode-jupyter-slideshow', 'ms-vscode-remote.remote-wsl', 'ms-vscode.vscode-speech', 'rangav.vscode-thunder-client', 'redhat.vscode-yaml', 'ritwickdey.lives

In [3]:
extensions_data = generate_extension_data(dotenv_path, extensions)
print(extensions_data)

                      extension name  \
0           Custom CSS and JS Loader   
1            GitHub Markdown Preview   
2                Markdown Checkboxes   
3                     Markdown Emoji   
4                 Markdown Footnotes   
5   Markdown Preview Mermaid Support   
6    Markdown Preview Github Styling   
7             Markdown yaml Preamble   
8               JetBrains Icon Theme   
9                       markdownlint   
10      GitLens â Git supercharged   
11                  Front Matter CMS   
12         Prettier - Code formatter   
13                    GitHub Copilot   
14               GitHub Copilot Chat   
15                      GitHub Theme   
16                                Go   
17                               SVG   
18                       JSON to CSV   
19                              JSON   
20                      Fluent Icons   
21                   Python Debugger   
22                            Python   
23                           Pylance   


In [5]:
save_as_csv = save_to_csv(extensions_data, 'extensions.csv')

In [4]:
save_as_md = save_to_markdown(extensions_data, 'extensions.md')